In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("movies.csv")
df1 = pd.read_csv("Movie_Id_Titles.csv")

In [3]:
df.head()

,user_id,item_id,rating,timestamp
0,0,50,5,881250949
1,0,172,5,881250949
2,0,133,1,881250949
3,196,242,3,881250949
4,186,302,3,891717742


In [4]:
df1.head()

,item_id,title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)


In [5]:
df1['title'] = df1.title.str.replace('(\(\d\d\d\d\))', '')
df1['title'] = df1['title'].apply(lambda x: x.strip())

C:\Users\HMERA\AppData\Local\Temp/ipykernel_3872/4131511800.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df1['title'] = df1.title.str.replace('(\(\d\d\d\d\))', '')


In [6]:
df1.head()

,item_id,title
0,1,Toy Story
1,2,GoldenEye
2,3,Four Rooms
3,4,Get Shorty
4,5,Copycat


In [7]:
df2 = pd.merge(df, df1, on= 'item_id')
df2.head()

,user_id,item_id,rating,timestamp,title
0,0,50,5,881250949,Star Wars
1,290,50,5,880473582,Star Wars
2,79,50,4,891271545,Star Wars
3,2,50,5,888552084,Star Wars
4,8,50,5,879362124,Star Wars


In [8]:
df2.drop(columns='timestamp', inplace=True, axis =1)
df2

,user_id,item_id,rating,title
0,0,50,5,Star Wars
1,290,50,5,Star Wars
2,79,50,4,Star Wars
3,2,50,5,Star Wars
4,8,50,5,Star Wars
...,...,...,...,...
99998,840,1674,4,Mamma Roma
99999,655,1640,3,"Eighth Day, The"
100000,655,1637,3,Girls Town
100001,655,1630,3,"Silence of the Palace, The (Saimt el Qusur)"


In [9]:
ratings =pd.DataFrame(df2.groupby('title')['rating'].mean())
ratings.head()

,rating
title,
'Til There Was You,2.333333
1-900,2.600000
101 Dalmatians,2.908257
12 Angry Men,4.344000
187,3.024390


In [10]:
ratings['rating_numbers'] = pd.DataFrame(df2.groupby('title')['rating'].count())
ratings.head()

,rating,rating_numbers
title,,
'Til There Was You,2.333333,9
1-900,2.600000,5
101 Dalmatians,2.908257,109
12 Angry Men,4.344000,125
187,3.024390,41


In [11]:
table = df2.pivot_table(index= 'user_id', columns= 'title', values= 'rating')
table.fillna(0, inplace= True)

In [12]:
table.head()

title,'Til There Was You,1-900,101 Dalmatians,12 Angry Men,187,2 Days in the Valley,"20,000 Leagues Under the Sea",2001: A Space Odyssey,3 Ninjas: High Noon At Mega Mountain,"39 Steps, The",...,Yankee Zulu,Year of the Horse,You So Crazy,Young Frankenstein,Young Guns,Young Guns II,"Young Poisoner's Handbook, The",Zeus and Roxanne,unknown,Á köldum klaka (Cold Fever)
user_id,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,2.0,5.0,0.0,0.0,3.0,4.0,0.0,0.0,...,0.0,0.0,0.0,5.0,3.0,0.0,0.0,0.0,4.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
def recommendMovies(name, min_rating_count = 50):
    user_rating = table[name]
    similar_movies = table.corrwith(user_rating)
    corr_movies = pd.DataFrame(similar_movies, columns= ['Correlation'])
    corr_movies.dropna(inplace= True)
    corr_movies = corr_movies.join(ratings['rating_numbers'], how='left', lsuffix='_left', rsuffix='_right')
    final = corr_movies[corr_movies['rating_numbers']>min_rating_count].sort_values('Correlation', ascending=False)
    return final.head(5)

In [15]:
recommendations = recommendMovies('Girls Town')
recommendations.index

Index(['In the Company of Men', 'Private Benjamin', 'Apostle, The', 'Bananas',
       'She's So Lovely'],
      dtype='object', name='title')

In [15]:
import pickle

In [43]:
pickle.dump(df2, open('movies.pkl', 'wb'))

In [47]:
df2['title'].values

array(['Star Wars', 'Star Wars', 'Star Wars', ..., 'Girls Town',
       'Silence of the Palace, The (Saimt el Qusur)', 'Dadetown'],
      dtype=object)

In [50]:
pickle.dump(df2.to_dict(), open('movie_dict.pkl', 'wb'))

In [20]:
pickle.dump(table, open('table.pkl', 'wb'))

In [21]:
pickle.dump(table.to_dict(), open('dict_table.pkl', 'wb'))

In [22]:
pickle.dump(ratings, open('ratings.pkl', 'wb'))

In [22]:
df1[df1['title'] == 'African Queen, The']

,item_id,title
497,498,"African Queen, The"
